In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter as ctr
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mac/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
df = pd.read_csv('Symptom2Disease.csv')
df.head(10)

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."
5,5,Psoriasis,The skin on my palms and soles is thickened an...
6,6,Psoriasis,"The skin around my mouth, nose, and eyes is re..."
7,7,Psoriasis,My skin is very sensitive and reacts easily to...
8,8,Psoriasis,I have noticed a sudden peeling of skin at dif...
9,9,Psoriasis,The skin on my genitals is red and inflamed. I...


In [4]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [5]:
ctr(df['label'])

Counter({'Psoriasis': 50,
         'Varicose Veins': 50,
         'Typhoid': 50,
         'Chicken pox': 50,
         'Impetigo': 50,
         'Dengue': 50,
         'Fungal infection': 50,
         'Common Cold': 50,
         'Pneumonia': 50,
         'Dimorphic Hemorrhoids': 50,
         'Arthritis': 50,
         'Acne': 50,
         'Bronchial Asthma': 50,
         'Hypertension': 50,
         'Migraine': 50,
         'Cervical spondylosis': 50,
         'Jaundice': 50,
         'Malaria': 50,
         'urinary tract infection': 50,
         'allergy': 50,
         'gastroesophageal reflux disease': 50,
         'drug reaction': 50,
         'peptic ulcer disease': 50,
         'diabetes': 50})

In [6]:
df.sample(10)

,label,text
961,allergy,I often sneeze and have a sore throat. My lips...
903,urinary tract infection,I have been getting blood in my pee. Sometimes...
233,Impetigo,"I'm running a high temperature. Near my nose, ..."
108,Typhoid,I've been having a lot of difficulty breathing...
549,Arthritis,"Lately, I've been having stiffness and weaknes..."
287,Dengue,I have been feeling nauseous and have a consta...
85,Varicose Veins,The veins on my legs are causing a lot of disc...
90,Varicose Veins,I have a rash on my legs that is giving me a l...
1027,gastroesophageal reflux disease,I frequently get a scratchy voice or a painful...
874,Malaria,"I've had intense itching, chills, vomiting, an..."


In [7]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    snowball_stemmer = SnowballStemmer('english')
    tokens = [snowball_stemmer.stem(token.lower()) for token in tokens if token.isalpha()]
    return ' '.join(tokens)

df['text'] = df['text'].apply(preprocess_text)

In [8]:
df.sample(5)

,label,text
1081,drug reaction,i have put on a lot of weight and am now veri ...
1174,diabetes,i have troubl focus and my emot fluctuat my br...
634,Bronchial Asthma,i been have a tough time with this constant co...
919,urinary tract infection,my stomach hurt and i frequent acquir a temper...
316,Fungal infection,i have been experienc some realli intens itch ...


In [9]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['text'])

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
base_models = [
    ('nb', MultinomialNB()),
    ('rf', RandomForestClassifier()),
    ('lr', LogisticRegression()),
    ('svm', SVC(kernel='linear', probability=True))
]

In [12]:
voting_classifier = VotingClassifier(estimators=base_models, voting='hard')

In [13]:
voting_classifier.fit(X_train, y_train)

VotingClassifier(estimators=[('nb', MultinomialNB()),
                             ('rf', RandomForestClassifier()),
                             ('lr', LogisticRegression()),
                             ('svm', SVC(kernel='linear', probability=True))])

In [14]:
accuracy = voting_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9708333333333333


In [15]:
import joblib

joblib.dump(voting_classifier, 'voting_classifier_model_Disease_pred_97_percent_acc.pkl')

['voting_classifier_model_Disease_pred_97_percent_acc.pkl']

In [16]:
loaded_model = joblib.load('voting_classifier_model_Disease_pred_97_percent_acc.pkl')

In [17]:
# Sample text
sample_text = "I have been experiencing a skin rash on my arm for the past few weeks."
sample_text_processed = preprocess_text(sample_text)
sample_text_transformed = tfidf_vectorizer.transform([sample_text_processed])
predicted_label = label_encoder.inverse_transform(voting_classifier.predict(sample_text_transformed))

print("Predicted Label:", predicted_label)

Predicted Label: ['Psoriasis']


In [18]:
text = 'i been realli weari and ill i been suffer from..'

sample_text = text
sample_text_processed = preprocess_text(sample_text)
sample_text_transformed = tfidf_vectorizer.transform([sample_text_processed])
predicted_label = label_encoder.inverse_transform(voting_classifier.predict(sample_text_transformed))

print("Predicted Label:", predicted_label)

Predicted Label: ['Common Cold']


In [19]:
df.sample()

,label,text
98,Varicose Veins,i am experienc too mani cramp in the last coup...


In [20]:
# Compute accuracy
accuracy = accuracy_score(y_test, y_test)
print("Accuracy:", accuracy)

# Compute precision
precision = precision_score(y_test, y_test, average='macro')  # 'macro' computes precision for each label, and returns the average
print("Precision:", precision)

# Compute recall
recall = recall_score(y_test, y_test, average='macro')  # 'macro' computes recall for each label, and returns the average
print("Recall:", recall)

# Compute F1-score
f1 = f1_score(y_test, y_test, average='macro')  # 'macro' computes F1-score for each label, and returns the average
print("F1-score:", f1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_test)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0
Confusion Matrix:
[[ 7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  

In [21]:
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer_disease_nlp.joblib')
joblib.dump(label_encoder, 'label_encoder_disease_nlp.joblib')

['label_encoder_disease_nlp.joblib']

In [22]:
import joblib
voting_classifier = joblib.load('voting_classifier_model_Disease_pred_97_percent_acc.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer_disease_nlp.joblib')
label_encoder = joblib.load('label_encoder_disease_nlp.joblib')

In [ ]:
import joblib
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

# Load the model and vectorizer
voting_classifier = joblib.load('voting_classifier_model_Disease_pred_97_percent_acc.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer_disease_nlp.joblib')
label_encoder = joblib.load('label_encoder_disease_nlp.joblib')

# Sample text
sample_text = input("Enter your symptoms: ")

# Preprocess the sample text
def preprocess_text(text):
    tokens = word_tokenize(text)
    snowball_stemmer = SnowballStemmer('english')
    tokens = [snowball_stemmer.stem(token.lower()) for token in tokens if token.isalpha()]
    return ' '.join(tokens)

sample_text_processed = preprocess_text(sample_text)

# Transform the preprocessed sample text using the loaded vectorizer
sample_text_transformed = tfidf_vectorizer.transform([sample_text_processed])

# Predict using the loaded model
predicted_label_encoded = voting_classifier.predict(sample_text_transformed)

# Decode the predicted label
predicted_label = label_encoder.inverse_transform(predicted_label_encoded)[0]
print("Predicted Label:", predicted_label)

# Define a dictionary mapping diseases to recommended tests and medications
disease_info = {
    "Psoriasis": {
        "tests": ["Skin biopsy", "Blood tests"],
        "medications": ["Neem", "Turmeric", "Aloe Vera", "Manjistha", "Guggulu"]
    },
    "Varicose Veins": {
        "tests": ["Doppler ultrasound", "Venography"],
        "medications": ["Horse chestnut", "Triphala", "Guggulu", "Brahmi", "Ashwagandha"]
    },
    "Typhoid": {
        "tests": ["Blood culture", "Widal test"],
        "medications": ["Giloy", "Neem", "Kutaj", "Triphala", "Chirayata"]
    },
    "Chickenpox": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Neem", "Tulsi", "Amla", "Sandalwood", "Licorice"]
    },
    "Impetigo": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Neem", "Manjistha", "Turmeric", "Triphala"]
    },
    "Dengue": {
        "tests": ["Dengue NS1 antigen test", "Dengue antibody test"],
        "medications": ["Papaya leaf extract", "Giloy", "Tulsi", "Turmeric", "Amla"]
    },
    "Fungal Infection": {
        "tests": ["Skin scraping", "KOH test"],
        "medications": ["Neem", "Manjistha", "Aloe Vera", "Turmeric", "Triphala"]
    },
    "Common Cold": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Ginger", "Tulsi", "Cinnamon", "Turmeric", "Honey"]
    },
    "Pneumonia": {
        "tests": ["Chest X-ray", "Blood tests"],
        "medications": ["Tulsi", "Pippali", "Trikatu", "Licorice", "Guduch s i"]
    },
    "Arthritis": {
        "tests": ["X-ray", "Blood tests"],
        "medications": ["Ashwagandha", "Guggulu", "Shallaki", "Turmeric", "Ginger"]
    },
    "Acne": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Neem", "Tulsi", "Aloe Vera", "Turmeric", "Manjistha"]
    },
    "Bronchial Asthma": {
        "tests": ["Spirometry", "Peak flow meter"],
        "medications": ["Vasaka", "Trikatu", "Tulsi", "Turmeric", "Ginger"]
    },
    "Hypertension": {
        "tests": ["Blood pressure measurement", "Blood tests"],
        "medications": ["Arjuna", "Ashwagandha", "Jatamansi", "Brahmi", "Gokshura"]
    },
    "Migraine": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Lavender oil", "Peppermint oil", "Ginger", "Turmeric", "Feverfew"]
    },
    "Cervical Spondylosis": {
        "tests": ["X-ray", "MRI"],
        "medications": ["Ashwagandha", "Guggulu", "Bala", "Shallaki", "Ginger"]
    },
    "Jaundice": {
        "tests": ["Liver function tests", "Bilirubin test"],
        "medications": ["Kutki", "Bhumiamla", "Triphala", "Punarnava", "Guduchi"]
    },
    "Malaria": {
        "tests": ["Blood smear", "Rapid diagnostic test"],
        "medications": ["Giloy", "Neem", "Tulsi", "Chirayata", "Quinine"]
    },
    "Urinary Tract Infection": {
        "tests": ["Urine analysis", "Urine culture"],
        "medications": ["Punarnava", "Gokshura", "Neem", "Manjistha", "Triphala"]
    },
    "Allergy": {
        "tests": ["Skin prick test", "Blood tests"],
        "medications": ["Neem", "Turmeric", "Ginger", "Tulsi", "Guduchi"]
    },
    "Gastroesophageal Reflux Disease": {
        "tests": ["Endoscopy", "Esophageal manometry"],
        "medications": ["Aloe Vera", "Licorice", "Tulsi", "Ginger", "Peppermint oil"]
    },
    "Drug Reaction": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Neem", "Tulsi", "Aloe Vera", "Manjistha", "Turmeric"]
    },
    "Peptic Ulcer Disease": {
        "tests": ["Endoscopy", "Urea breath test"],
        "medications": ["Licorice", "Aloe Vera", "Neem", "Tulsi", "Ginger"]
    },
    "Diabetes": {
        "tests": ["Fasting blood sugar test", "Oral glucose tolerance test"],
        "medications": ["Gudmar", "Fenugreek", "Turmeric", "Amla", "Neem"]
    },
    
}
def get_recommendations(predicted_label):
    if predicted_label in disease_info:
        return disease_info[predicted_label]["tests"], disease_info[predicted_label]["medications"]
    else:
        return [], []

recommended_tests, recommended_medications = get_recommendations(predicted_label)
print("Recommended Tests:", recommended_tests)
print("Recommended Medications:", recommended_medications)
print("Recommended Medications:", recommended_medications

# Function to predict disease from user input image
def predict_disease(image_path):
    # Preprocess the image
    img = preprocess_image(image_path)
    
    # Make predictions
    prediction = model.predict(img)
    
    # Get the predicted class index
    predicted_class_index = np.argmax(prediction)
    
    # Map the predicted class index to the corresponding class label
    class_labels = train_generator.class_indices
    predicted_disease = [key for key, value in class_labels.items() if value == predicted_class_index][0]
    
    return predicted_disease

# Get user input image path
user_image_path = input("Enter the path of the image: ")

# Predict disease for user input image
predicted_disease = predict_disease(user_image_path)

# Display prediction

print("Predicted disease:", predicted_disease)


In [ ]:
import joblib
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import tkinter as tk
from tkinter import scrolledtext, messagebox
from PIL import ImageTk, Image
import numpy as np

# Load the disease prediction model and vectorizer
voting_classifier = joblib.load('voting_classifier_model_Disease_pred_97_percent_acc.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer_disease_nlp.joblib')
label_encoder = joblib.load('label_encoder_disease_nlp.joblib')

# Load the CNN model for image analysis (replace 'your_cnn_model.h5' with the path to your CNN model file)
# Assuming your CNN model is loaded using Keras
from keras.models import load_model
cnn_model = load_model('resnet.h5')

# Define a dictionary mapping diseases to recommended tests and medications
disease_info = {
    "Psoriasis": {
        "tests": ["Skin biopsy", "Blood tests"],
        "medications": {
            "Neem": {"dosage": "1-2 tsp of powder or oil", "timing": "twice daily"},
            "Turmeric": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Aloe Vera": {"dosage": "2-3 tbsp of gel", "timing": "as needed"},
            "Manjistha": {"dosage": "500 mg", "timing": "twice daily"},
            "Guggulu": {"dosage": "500 mg", "timing": "twice daily"}
        }
    },
    "Varicose Veins": {
        "tests": ["Doppler ultrasound", "Venography"],
        "medications": {
            "Horse chestnut": {"dosage": "300 mg", "timing": "twice daily"},
            "Triphala": {"dosage": "1 tsp", "timing": "once daily"},
            "Guggulu": {"dosage": "500 mg", "timing": "twice daily"},
            "Brahmi": {"dosage": "300 mg", "timing": "once daily"},
            "Ashwagandha": {"dosage": "500 mg", "timing": "twice daily"}
        }
    },
    "Typhoid": {
        "tests": ["Blood culture", "Widal test"],
        "medications": {
            "Giloy": {"dosage": "500 mg", "timing": "twice daily"},
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "once daily"},
            "Kutaj": {"dosage": "500 mg", "timing": "twice daily"},
            "Triphala": {"dosage": "1 tsp", "timing": "once daily"},
            "Chirayata": {"dosage": "500 mg", "timing": "twice daily"}
        }
    },
    "Chickenpox": {
        "tests": ["Clinical diagnosis"],
        "medications": {
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"},
            "Tulsi": {"dosage": "5-10 leaves", "timing": "once daily"},
            "Amla": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Sandalwood": {"dosage": "as needed for topical use", "timing": "as needed"},
            "Licorice": {"dosage": "1 tsp of powder", "timing": "twice daily"}
        }
    },
    "Impetigo": {
        "tests": ["Clinical diagnosis"],
        "medications": {
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"},
            "Manjistha": {"dosage": "500 mg", "timing": "twice daily"},
            "Turmeric": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Triphala": {"dosage": "1 tsp", "timing": "once daily"}
        }
    },
    "Dengue": {
        "tests": ["Dengue NS1 antigen test", "Dengue antibody test"],
        "medications": {
            "Papaya leaf extract": {"dosage": "50 ml", "timing": "twice daily"},
            "Giloy": {"dosage": "500 mg", "timing": "twice daily"},
            "Tulsi": {"dosage": "5-10 leaves", "timing": "once daily"},
            "Turmeric": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Amla": {"dosage": "1 tsp of powder", "timing": "once daily"}
        }
    },
    "Fungal Infection": {
        "tests": ["Skin scraping", "KOH test"],
        "medications": {
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"},
            "Manjistha": {"dosage": "500 mg", "timing": "twice daily"},
            "Aloe Vera": {"dosage": "2-3 tbsp of gel", "timing": "as needed"},
            "Turmeric": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Triphala": {"dosage": "1 tsp", "timing": "once daily"}
        }
    },
    "Common Cold": {
        "tests": ["Clinical diagnosis"],
        "medications": {
            "Ginger": {"dosage": "1 inch piece", "timing": "as needed"},
            "Tulsi": {"dosage": "5-10 leaves", "timing": "twice daily"},
            "Cinnamon": {"dosage": "1/2 tsp", "timing": "once daily"},
            "Turmeric": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Honey": {"dosage": "1 tsp", "timing": "as needed"}
        }
    },
    "Pneumonia": {
        "tests": ["Chest X-ray", "Blood tests"],
        "medications": {
            "Tulsi": {"dosage": "5-10 leaves", "timing": "twice daily"},
            "Pippali": {"dosage": "500 mg", "timing": "twice daily"},
            "Trikatu": {"dosage": "500 mg", "timing": "twice daily"},
            "Licorice": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Guduchi": {"dosage": "500 mg", "timing": "twice daily"}
        }
    },
    "Arthritis": {
        "tests": ["X-ray", "Blood tests"],
        "medications": {
            "Ashwagandha": {"dosage": "500 mg", "timing": "twice daily"},
            "Guggulu": {"dosage": "500 mg", "timing": "twice daily"},
            "Shallaki": {"dosage": "500 mg", "timing": "twice daily"},
            "Turmeric": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Ginger": {"dosage": "1 inch piece", "timing": "as needed"}
        }
    },
    "Acne": {
        "tests": ["Clinical diagnosis"],
        "medications": {
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"},
            "Tulsi": {"dosage": "5-10 leaves", "timing": "once daily"},
            "Aloe Vera": {"dosage": "2-3 tbsp of gel", "timing": "as needed"},
            "Turmeric": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Manjistha": {"dosage": "500 mg", "timing": "twice daily"}
        }
    },
    "Bronchial Asthma": {
        "tests": ["Spirometry", "Peak flow meter"],
        "medications": {
            "Vasaka": {"dosage": "500 mg", "timing": "twice daily"},
            "Trikatu": {"dosage": "500 mg", "timing": "twice daily"},
            "Tulsi": {"dosage": "5-10 leaves", "timing": "twice daily"},
            "Turmeric": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Ginger": {"dosage": "1 inch piece", "timing": "as needed"}
        }
    },
    "Hypertension": {
        "tests": ["Blood pressure measurement", "Blood tests"],
        "medications": {
            "Arjuna": {"dosage": "500 mg", "timing": "twice daily"},
            "Ashwagandha": {"dosage": "500 mg", "timing": "twice daily"},
            "Jatamansi": {"dosage": "500 mg", "timing": "once daily"},
            "Brahmi": {"dosage": "300 mg", "timing": "once daily"},
            "Gokshura": {"dosage": "500 mg", "timing": "twice daily"}
        }
    },
    "Migraine": {
        "tests": ["Clinical diagnosis"],
        "medications": {
            "Lavender oil": {"dosage": "2-3 drops", "timing": "as needed"},
            "Peppermint oil": {"dosage": "2-3 drops", "timing": "as needed"},
            "Ginger": {"dosage": "1 inch piece", "timing": "as needed"},
            "Turmeric": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Feverfew": {"dosage": "500 mg", "timing": "twice daily"}
        }
    },
    "Cervical Spondylosis": {
        "tests": ["X-ray", "MRI"],
        "medications": {
            "Ashwagandha": {"dosage": "500 mg", "timing": "twice daily"},
            "Guggulu": {"dosage": "500 mg", "timing": "twice daily"},
            "Bala": {"dosage": "500 mg", "timing": "once daily"},
            "Shallaki": {"dosage": "500 mg", "timing": "twice daily"},
            "Ginger": {"dosage": "1 inch piece", "timing": "as needed"}
        }
    },
    "Jaundice": {
        "tests": ["Liver function tests", "Bilirubin test"],
        "medications": {
            "Kutki": {"dosage": "500 mg", "timing": "twice daily"},
            "Bhumiamla": {"dosage": "500 mg", "timing": "twice daily"},
            "Triphala": {"dosage": "1 tsp", "timing": "once daily"},
            "Punarnava": {"dosage": "500 mg", "timing": "twice daily"},
            "Guduchi": {"dosage": "500 mg", "timing": "twice daily"}
        }
    },
    "Malaria": {
        "tests": ["Blood smear", "Rapid diagnostic test"],
        "medications": {
            "Giloy": {"dosage": "500 mg", "timing": "twice daily"},
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"},
            "Tulsi": {"dosage": "5-10 leaves", "timing": "once daily"},
            "Chirayata": {"dosage": "500 mg", "timing": "twice daily"},
            "Quinine": {"dosage": "500 mg", "timing": "as prescribed"}
        }
    },
    "Urinary Tract Infection": {
        "tests": ["Urine analysis", "Urine culture"],
        "medications": {
            "Punarnava": {"dosage": "500 mg", "timing": "twice daily"},
            "Gokshura": {"dosage": "500 mg", "timing": "twice daily"},
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"},
            "Manjistha": {"dosage": "500 mg", "timing": "twice daily"},
            "Triphala": {"dosage": "1 tsp", "timing": "once daily"}
        }
    },
    "Allergy": {
        "tests": ["Skin prick test", "Blood tests"],
        "medications": {
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"},
            "Turmeric": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Ginger": {"dosage": "1 inch piece", "timing": "as needed"},
            "Tulsi": {"dosage": "5-10 leaves", "timing": "twice daily"},
            "Guduchi": {"dosage": "500 mg", "timing": "twice daily"}
        }
    },
    "Gastroesophageal Reflux Disease": {
        "tests": ["Endoscopy", "Esophageal manometry"],
        "medications": {
            "Aloe Vera": {"dosage": "2-3 tbsp of gel", "timing": "as needed"},
            "Licorice": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Tulsi": {"dosage": "5-10 leaves", "timing": "twice daily"},
            "Ginger": {"dosage": "1 inch piece", "timing": "as needed"},
            "Peppermint oil": {"dosage": "2-3 drops", "timing": "as needed"}
        }
    },
    "Drug Reaction": {
        "tests": ["Clinical diagnosis"],
        "medications": {
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"},
            "Tulsi": {"dosage": "5-10 leaves", "timing": "twice daily"},
            "Aloe Vera": {"dosage": "2-3 tbsp of gel", "timing": "as needed"},
            "Manjistha": {"dosage": "500 mg", "timing": "twice daily"},
            "Turmeric": {"dosage": "1 tsp of powder", "timing": "once daily"}
        }
    },
    "Peptic Ulcer Disease": {
        "tests": ["Endoscopy", "Urea breath test"],
        "medications": {
            "Licorice": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Aloe Vera": {"dosage": "2-3 tbsp of gel", "timing": "as needed"},
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"},
            "Tulsi": {"dosage": "5-10 leaves", "timing": "once daily"},
            "Ginger": {"dosage": "1 inch piece", "timing": "as needed"}
        }
    },
    "Diabetes": {
        "tests": ["Fasting blood sugar test", "Oral glucose tolerance test"],
        "medications": {
            "Gudmar": {"dosage": "500 mg", "timing": "twice daily"},
            "Fenugreek": {"dosage": "1 tsp", "timing": "once daily"},
            "Turmeric": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Amla": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"}
        }
    }
}

# Function to preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text)
    snowball_stemmer = SnowballStemmer('english')
    tokens = [snowball_stemmer.stem(token.lower()) for token in tokens if token.isalpha()]
    return ' '.join(tokens)

# Function to preprocess image
def preprocess_image(image_path):
    # Add preprocessing steps as needed for your CNN model
    img = Image.open(image_path)
    # Resize the image to match the input size of your CNN model
    img = img.resize((224, 224))
    # Convert the image to array
    img_array = np.array(img)
    # Perform any other preprocessing steps required by your CNN model
    return img_array

# Function to predict disease and get recommendations
def predict_disease_and_recommendations(symptoms):
    # Preprocess the symptoms
    symptoms_processed = preprocess_text(symptoms)
    
    # Transform the preprocessed text using the loaded vectorizer
    symptoms_transformed = tfidf_vectorizer.transform([symptoms_processed])
    
    # Predict using the loaded model
    predicted_label_encoded = voting_classifier.predict(symptoms_transformed)
    
    # Decode the predicted label
    predicted_label = label_encoder.inverse_transform(predicted_label_encoded)[0]
    
    # Get recommended tests and medications for the predicted disease
    if predicted_label in disease_info:
        recommended_tests = ", ".join(disease_info[predicted_label]["tests"])
        recommended_medications = ", ".join(disease_info[predicted_label]["medications"])
        
    else:
        recommended_tests = "Not available"
        recommended_medications = "Not available"
    
    return predicted_label, recommended_tests, recommended_medications




In [5]:
import joblib
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import tkinter as tk
from tkinter import scrolledtext, messagebox, filedialog
from PIL import ImageTk, Image
import numpy as np
import pandas as pd
from keras.models import load_model

# Load models
voting_classifier = joblib.load('voting_classifier_model_Disease_pred_97_percent_acc.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer_disease_nlp.joblib')
label_encoder = joblib.load('label_encoder_disease_nlp.joblib')
cnn_model = load_model('resnet.h5')

# Disease information dictionary
disease_info = {
    "Psoriasis": {
        "tests": ["Skin biopsy", "Blood tests"],
        "medications": {
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"}
        }
    },
    "Diabetes": {
        "tests": ["Fasting blood sugar test", "Oral glucose tolerance test"],
        "medications": {
            "Gudmar": {"dosage": "500 mg", "timing": "twice daily"}
        }
    },
    "Chickenpox": {
        "tests": ["Clinical diagnosis"],
        "medications": {
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"},
            "Tulsi": {"dosage": "5-10 leaves", "timing": "once daily"},
            "Amla": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Sandalwood": {"dosage": "as needed for topical use", "timing": "as needed"},
            "Licorice": {"dosage": "1 tsp of powder", "timing": "twice daily"}
        }
    }
}

# Function to preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text)
    snowball_stemmer = SnowballStemmer('english')
    tokens = [snowball_stemmer.stem(token.lower()) for token in tokens if token.isalpha()]
    return ' '.join(tokens)

# Function to preprocess image
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))
    img_array = np.array(img) / 255.0  # Normalize
    return img_array

# Function to predict disease and get recommendations
def predict_disease_and_recommendations(symptoms):
    symptoms_processed = preprocess_text(symptoms)
    symptoms_transformed = tfidf_vectorizer.transform([symptoms_processed])
    predicted_label_encoded = voting_classifier.predict(symptoms_transformed)
    predicted_label = label_encoder.inverse_transform(predicted_label_encoded)[0]

    if predicted_label in disease_info:
        recommended_tests = ", ".join(disease_info[predicted_label]["tests"])
        recommended_medications = "\n".join([
            f"{med}: {details['dosage']} ({details['timing']})"
            for med, details in disease_info[predicted_label]["medications"].items()
        ])
    else:
        recommended_tests = "Not available"
        recommended_medications = "Not available"
    
    return predicted_label, recommended_tests, recommended_medications

# Function to handle the upload of an image
def upload_image():
    file_path = filedialog.askopenfilename(title="Select an Image", filetypes=[("Image files", "*.jpg *.jpeg *.png")])
    if file_path:
        img = Image.open(file_path)
        img = img.resize((150, 150))  # Resize for display
        img_tk = ImageTk.PhotoImage(img)
        image_label.config(image=img_tk)
        image_label.image = img_tk  # Keep a reference to avoid garbage collection

# Function to handle predictions
def handle_predict():
    symptoms = symptoms_text.get("1.0", tk.END).strip()
    if not symptoms:
        messagebox.showerror("Input Error", "Please enter symptoms.")
        return
    
    disease, tests, medications = predict_disease_and_recommendations(symptoms)
    result_text.set(f"Disease Predicted: {disease}\nRecommended Tests: {tests}\nRecommended Medications:\n{medications}")

    # Save patient data (symptoms, predictions)
    patient_data = {
        "Symptoms": symptoms,
        "Predicted Disease": disease,
        "Recommended Tests": tests,
        "Recommended Medications": medications
    }
    df = pd.DataFrame([patient_data])
    df.to_csv('patient_data.csv', mode='a', header=False, index=False)

# Setting up the GUI
root = tk.Tk()
root.title("Medsynapse - Skin Disease Prediction System")

# Input Area
tk.Label(root, text="Enter Symptoms:").pack()
symptoms_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=50, height=10)
symptoms_text.pack()

# Upload Image Button
upload_button = tk.Button(root, text="Upload Image", command=upload_image)
upload_button.pack()

# Image Display
image_label = tk.Label(root)
image_label.pack()

# Predict Button
predict_button = tk.Button(root, text="Predict Disease", command=handle_predict)
predict_button.pack()

# Result Area
result_text = tk.StringVar()
result_label = tk.Label(root, textvariable=result_text, wraplength=400)
result_label.pack()

# Run the application
root.mainloop()


In [ ]:
import joblib
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import tkinter as tk
from tkinter import scrolledtext, messagebox, filedialog
from PIL import ImageTk, Image
import numpy as np
import pandas as pd
from keras.models import load_model

# Load models
voting_classifier = joblib.load('voting_classifier_model_Disease_pred_97_percent_acc.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer_disease_nlp.joblib')
label_encoder = joblib.load('label_encoder_disease_nlp.joblib')
cnn_model = load_model('resnet.h5')

# Disease information dictionary
disease_info = {
    "Psoriasis": {
        "tests": ["Skin biopsy", "Blood tests"],
        "medications": {
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"}
        }
    },
    "Diabetes": {
        "tests": ["Fasting blood sugar test", "Oral glucose tolerance test"],
        "medications": {
            "Gudmar": {"dosage": "500 mg", "timing": "twice daily"}
        }
    },
    "Chickenpox": {
        "tests": ["Clinical diagnosis"],
        "medications": {
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"},
            "Tulsi": {"dosage": "5-10 leaves", "timing": "once daily"},
            "Amla": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Sandalwood": {"dosage": "as needed for topical use", "timing": "as needed"},
            "Licorice": {"dosage": "1 tsp of powder", "timing": "twice daily"}
        }
    },
    "Impetigo": {
        "tests": ["Clinical diagnosis", "Swab test"],
        "medications": {
            "Mupirocin Ointment": {"dosage": "apply to affected area", "timing": "three times daily for 5 days"},
            "Cephalexin": {"dosage": "500 mg", "timing": "three times daily for 7 days"},
            "Clindamycin": {"dosage": "300 mg", "timing": "three times daily for 10 days"}
        }
    }
}

# Function to preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text)
    snowball_stemmer = SnowballStemmer('english')
    tokens = [snowball_stemmer.stem(token.lower()) for token in tokens if token.isalpha()]
    return ' '.join(tokens)

# Function to predict disease and get recommendations
def predict_disease_and_recommendations(symptoms):
    symptoms_processed = preprocess_text(symptoms)
    symptoms_transformed = tfidf_vectorizer.transform([symptoms_processed])
    predicted_label_encoded = voting_classifier.predict(symptoms_transformed)
    predicted_label = label_encoder.inverse_transform(predicted_label_encoded)[0]

    if predicted_label in disease_info:
        recommended_tests = ", ".join(disease_info[predicted_label]["tests"])
        recommended_medications = "\n".join([
            f"{med}: {details['dosage']} ({details['timing']})"
            for med, details in disease_info[predicted_label]["medications"].items()
        ])
    else:
        recommended_tests = "Not available"
        recommended_medications = "Not available"
    
    return predicted_label, recommended_tests, recommended_medications

# Function to handle the upload of an image
def upload_image():
    file_path = filedialog.askopenfilename(title="Select an Image", filetypes=[("Image files", "*.jpg *.jpeg *.png")])
    if file_path:
        img = Image.open(file_path)
        img = img.resize((150, 150))  # Resize for display
        img_tk = ImageTk.PhotoImage(img)
        image_label.config(image=img_tk)
        image_label.image = img_tk  # Keep a reference to avoid garbage collection

# Function to handle predictions
def handle_predict():
    symptoms = symptoms_text.get("1.0", tk.END).strip()
    if not symptoms:
        messagebox.showerror("Input Error", "Please enter symptoms.")
        return
    
    disease, tests, medications = predict_disease_and_recommendations(symptoms)
    result_text.set(f"Disease Predicted: {disease}\nRecommended Tests: {tests}\nRecommended Medications:\n{medications}")

    # Save patient data (symptoms, predictions)
    patient_data = {
        "Symptoms": symptoms,
        "Predicted Disease": disease,
        "Recommended Tests": tests,
        "Recommended Medications": medications
    }
    df = pd.DataFrame([patient_data])
    df.to_csv('patient_data.csv', mode='a', header=False, index=False)

# Setting up the GUI
root = tk.Tk()
root.title("Medsynapse - Skin Disease Prediction System")

# Input Area
tk.Label(root, text="Enter Symptoms:").pack()
symptoms_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=50, height=10)
symptoms_text.pack()

# Upload Image Button
upload_button = tk.Button(root, text="Upload Image", command=upload_image)
upload_button.pack()

# Image Display
image_label = tk.Label(root)
image_label.pack()

# Predict Button
predict_button = tk.Button(root, text="Predict Disease", command=handle_predict)
predict_button.pack()

# Result Area
result_text = tk.StringVar()
result_label = tk.Label(root, textvariable=result_text, wraplength=400)
result_label.pack()

# Medicine choice
medicine_choice = tk.StringVar(value="Allopathy")
tk.Label(root, text="Choose Medicine Type:").pack()
allopathy_radio = tk.Radiobutton(root, text="Allopathy", variable=medicine_choice, value="Allopathy")
allopathy_radio.pack()
ayurvedic_radio = tk.Radiobutton(root, text="Ayurvedic", variable=medicine_choice, value="Ayurvedic")
ayurvedic_radio.pack()

# Run the application
root.mainloop()


In [10]:
import joblib
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import tkinter as tk
from tkinter import scrolledtext, messagebox, filedialog
from PIL import ImageTk, Image
import numpy as np
import pandas as pd
from keras.models import load_model

# Load models
voting_classifier = joblib.load('voting_classifier_model_Disease_pred_97_percent_acc.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer_disease_nlp.joblib')
label_encoder = joblib.load('label_encoder_disease_nlp.joblib')
cnn_model = load_model('resnet.h5')

# Disease information dictionary (remains unchanged)
disease_info = {
    "Psoriasis": {
        "tests": ["Skin biopsy", "Blood tests"],
        "medications": {
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"}
        }
    },
    "Diabetes": {
        "tests": ["Fasting blood sugar test", "Oral glucose tolerance test"],
        "medications": {
            "Gudmar": {"dosage": "500 mg", "timing": "twice daily"}
        }
    },
    "Chickenpox": {
        "tests": ["Clinical diagnosis"],
        "medications": {
            "Neem": {"dosage": "1-2 tsp of powder", "timing": "twice daily"},
            "Tulsi": {"dosage": "5-10 leaves", "timing": "once daily"},
            "Amla": {"dosage": "1 tsp of powder", "timing": "once daily"},
            "Sandalwood": {"dosage": "as needed for topical use", "timing": "as needed"},
            "Licorice": {"dosage": "1 tsp of powder", "timing": "twice daily"}
        }
    },
    "Impetigo": {
        "tests": ["Clinical diagnosis", "Swab test"],
        "medications": {
            "Mupirocin Ointment": {"dosage": "apply to affected area", "timing": "three times daily for 5 days"},
            "Cephalexin": {"dosage": "500 mg", "timing": "three times daily for 7 days"},
            "Clindamycin": {"dosage": "300 mg", "timing": "three times daily for 10 days"}
        }
    }
}

# Load CSV files
#ayurvedic_df = pd.read_csv('aa.csv')
#allopathy_df = pd.read_csv('all.csv')

# Function to preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text)
    snowball_stemmer = SnowballStemmer('english')
    tokens = [snowball_stemmer.stem(token.lower()) for token in tokens if token.isalpha()]
    return ' '.join(tokens)

# Function to predict disease and get recommendations
def predict_disease_and_recommendations(symptoms, medicine_type):
    symptoms_processed = preprocess_text(symptoms)
    symptoms_transformed = tfidf_vectorizer.transform([symptoms_processed])
    predicted_label_encoded = voting_classifier.predict(symptoms_transformed)
    predicted_label = label_encoder.inverse_transform(predicted_label_encoded)[0]

    # Get recommendations based on the type of medicine
    if medicine_type == "Ayurvedic":
        recommended_row = ayurvedic_df[ayurvedic_df['Disease'] == predicted_label]
    else:  # Default to Allopathy
        recommended_row = allopathy_df[allopathy_df['Disease'] == predicted_label]

    if not recommended_row.empty:
        recommended_tests = recommended_row['Tests'].values[0]
        medications = recommended_row.drop(columns=['Disease', 'Tests']).to_dict(orient='records')[0]

        recommended_medications = "\n".join(
            f"{med}: {details['Dosage']} ({details['Timing']})"
            for med, details in medications.items() if pd.notna(details)
        )
    else:
        recommended_tests = "Not available"
        recommended_medications = "Not available"

    return predicted_label, recommended_tests, recommended_medications

# Function to handle the upload of an image
def upload_image():
    file_path = filedialog.askopenfilename(title="Select an Image", filetypes=[("Image files", "*.jpg *.jpeg *.png")])
    if file_path:
        img = Image.open(file_path)
        img = img.resize((150, 150))  # Resize for display
        img_tk = ImageTk.PhotoImage(img)
        image_label.config(image=img_tk)
        image_label.image = img_tk  # Keep a reference to avoid garbage collection

# Function to handle predictions
def handle_predict():
    symptoms = symptoms_text.get("1.0", tk.END).strip()
    if not symptoms:
        messagebox.showerror("Input Error", "Please enter symptoms.")
        return
    
    medicine_type = medicine_choice.get()
    disease, tests, medications = predict_disease_and_recommendations(symptoms, medicine_type)
    result_text.set(f"Disease Predicted: {disease}\nRecommended Tests: {tests}\nRecommended Medications:\n{medications}")

    # Save patient data (symptoms, predictions)
    patient_data = {
        "Symptoms": symptoms,
        "Predicted Disease": disease,
        "Recommended Tests": tests,
        "Recommended Medications": medications
    }
    df = pd.DataFrame([patient_data])
    df.to_csv('patient_data.csv', mode='a', header=False, index=False)

# Setting up the GUI
root = tk.Tk()
root.title("Medsynapse - Skin Disease Prediction System")

# Input Area
tk.Label(root, text="Enter Symptoms:").pack()
symptoms_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=50, height=10)
symptoms_text.pack()

# Medicine choice
medicine_choice = tk.StringVar(value="Allopathy")
tk.Label(root, text="Choose Medicine Type:").pack()
allopathy_radio = tk.Radiobutton(root, text="Allopathy", variable=medicine_choice, value="Allopathy")
allopathy_radio.pack()
ayurvedic_radio = tk.Radiobutton(root, text="Ayurvedic", variable=medicine_choice, value="Ayurvedic")
ayurvedic_radio.pack()

# Upload Image Button
upload_button = tk.Button(root, text="Upload Image", command=upload_image)
upload_button.pack()

# Image Display
image_label = tk.Label(root)
image_label.pack()

# Predict Button
predict_button = tk.Button(root, text="Predict Disease", command=handle_predict)
predict_button.pack()

# Result Area
result_text = tk.StringVar()
result_label = tk.Label(root, textvariable=result_text, wraplength=400)
result_label.pack()

# Run the application
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/80/jd0146wj1sn0f47f5phvs8t00000gn/T/ipykernel_22934/2003755460.py", line 107, in handle_predict
    disease, tests, medications = predict_disease_and_recommendations(symptoms, medicine_type)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/80/jd0146wj1sn0f47f5phvs8t00000gn/T/ipykernel_22934/2003755460.py", line 79, in predict_disease_and_recommendations
    recommended_medications = "\n".join(
                              ^^^^^^^^^^
  File "/var/folders/80/jd0146wj1sn0f47f5phvs8t00000gn/T/ipykernel_22934/2003755460.py", line 80, in <genexpr>
    f"{med}: {details['Dosage']} ({details['Timing']})"
              ~~~~~~~^^^^^^^^^^
TypeError: string indices must be integers, not 'str'
Exception in Tkin